## Header

In [ ]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id
# ------------------------------------------define GPU id/s to use
GPU_IDS = '0,1'
GPUS = choose_gpu_by_id(GPU_IDS)
print(GPUS)
# ------------------------------------------jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# ------------------------------------------ import helpers
# this should import glob, os, and some other standard libs to keep this cell clean
# local imports
from src.utils.Notebook_imports import *
from src.utils.Utils_io import Console_and_file_logger, init_config

# import external libs
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import cv2
import pandas as pd
import numpy as np
from ipyfilechooser import FileChooser
import nrrd as nrrd # https://pypi.org/project/pynrrd/
from src.data.Generators import DataGenerator

## Hella Sus Code that belongs to different notebooks I Think? 

In [ ]:
initial_epoch = 0

results = model.fit(
    x=batch_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks = get_callbacks(config, batch_generator,validation_generator),
    initial_epoch=initial_epoch,
    max_queue_size=12,
    verbose=1)

In [ ]:
# write the model summary to a txt file
# Open the file
with open(os.path.join(EXP_PATH, 'model_summary.txt') ,'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
    
tf.keras.utils.plot_model(
    model, show_shapes=True,
    to_file=os.path.join(EXP_PATH, 'model.png'),
    show_layer_names=True, 
    rankdir='TB', 
    expand_nested=True, dpi=96
)
    
#model.summary(line_length=150)

In [ ]:
# test if all batches are processable
"""for b in range(len(batch_generator)):
    print(b)
    for im in range(BATCHSIZE):
        select_image_in_batch(batch=b,im=im, slice_n=5, show_overview=True, show_input_vol=True)"""

In [ ]:
"""
load past config for model training 
"""
if 'streategy' in locals():
    pass
else:
    # distribute the training with the mirrored data paradigm across multiple gpus if available, if not use gpu 0
    strategy = tf.distribute.MirroredStrategy(devices=config.get('GPUS', ["/gpu:0"]))

import src.models.Unets as modelmanager
with strategy.scope():
    # create new model
    logging.info('Create model')
    model = modelmanager.create_unet(config, metrics, supervision=False)
    model.summary()

In [ ]:
# Load a previous config and by this a pre-trained model
from ipyfilechooser import FileChooser
config_chooser = FileChooser(os.path.join(os.getcwd(),'reports/configs'), 'config.json')
display(config_chooser)

## Config Loader

In [ ]:
exp_config_chooser = FileChooser(os.path.join(os.getcwd(),'exp/cv_baseline'), '')
display(exp_config_chooser)
@interact_manual
def load_config():

    global exp_config_chooser, config
    """
    load an experiment config
    """
    if 'exp_config_chooser' in globals():
        config_file  = exp_config_chooser.selected
    else:
        print('no config chooser found')

    # load the experiment config
    with open(config_file, encoding='utf-8') as data_file:
        config = json.loads(data_file.read())
    globals().update(config)
    Console_and_file_logger(EXPERIMENT, logging.INFO)
    logging.info('Loaded config for experiment: {}'.format(config['EXPERIMENT']))

## Generators

In [ ]:
# Load SAX volumes
from src.data.Dataset import get_trainings_files
x_train_sax, y_train_sax, x_val_sax, y_val_sax = get_trainings_files(data_path=DATA_PATH_SAX,
                                                                     path_to_folds_df=DF_FOLDS,
                                                                     fold=FOLD)
logging.info('SAX train CMR: {}, SAX train masks: {}'.format(len(x_train_sax), len(y_train_sax)))
logging.info('SAX val CMR: {}, SAX val masks: {}'.format(len(x_val_sax), len(y_val_sax)))

In [ ]:
# DataGenerator for SAX, changed from PhaseRegression Generator
# logging.getLogger().setLevel(logging.INFO)
from src.data.Generators import DataGenerator
config['SHUFFLE'] = False
config['AUGMENT'] = False
config['AUGMENT_GRID'] = False# make sure no augmentation will be applied to the validation data
config['HIST_MATCHING'] = False
config['BATCHSIZE'] = 1
batch_generator = DataGenerator(x_train_sax, y_train_sax, config=config)
# create another config for the validation data, 
# by this we can have a different set of parameters for both generators
val_config = config.copy()
validation_generator = DataGenerator(x_val_sax, y_val_sax, config=val_config)

In [ ]:
generator = validation_generator
@interact
def select_image_in_batch(batch = (0,len(generator)-1, 1), im = (0,config['BATCHSIZE']- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    global x, y
    x, y = generator.__getitem__(batch)
    info('selected batch : ' + str(batch))
    # logging level == debug --> visualise the generator steps
    info('X shape: {}, Y shape: {}'.format(x.shape, y.shape))
    show_2D_or_3D(x[im], y[im], interpol='bilinear',dpi=100,f_size=(5,5))
    plt.show()
    
    plt.hist(x[im].flatten())
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))

In [ ]:
generator = validation_generator
@interact
def select_image_in_batch(batch = (0,len(generator)-1, 1), im = (0,config['BATCHSIZE']- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    global x, y
    x, y = generator.__getitem__(batch)
    info('selected batch : ' + str(batch))
    # logging level == debug --> visualise the generator steps
    info('X shape: {}, Y shape: {}'.format(x.shape, y.shape))
    show_2D_or_3D(x[im], y[im], interpol='bilinear',dpi=100,f_size=(5,5))
    plt.show()
    
    plt.hist(x[im].flatten())
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))

In [ ]:
generator = validation_generator
@interact
def select_image_in_batch(batch = (0,len(generator)-1, 1), im = (0,config['BATCHSIZE']- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    global x, y
    x, y = generator.__getitem__(batch)
    info('selected batch : ' + str(batch))
    # logging level == debug --> visualise the generator steps
    info('X shape: {}, Y shape: {}'.format(x.shape, y.shape))
    show_2D_or_3D(x[im], y[im], interpol='bilinear',dpi=100,f_size=(5,5))
    plt.show()
    
    plt.hist(x[im].flatten())
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))

In [ ]:
# Load SAX volumes
x_train, y_train, x_val, y_val =  get_trainings_files(data_path=DATA_PATH_SAX,path_to_folds_df=DF_FOLDS, fold=FOLD)
logging.info('SAX train CMR: {}, SAX train masks: {}'.format(len(x_train), len(y_train)))
logging.info('SAX val CMR: {}, SAX val masks: {}'.format(len(x_val), len(y_val)))

# create a batch generator
batch_generator = DataGenerator(x_train, y_train, config=config)
val_config = config.copy()
val_config['AUGMENT_GRID'] = False# make sure no augmentation will be applied to the validation data
val_config['AUGMENT'] = False
val_config['HIST_MATCHING'] = False
validation_generator = DataGenerator(x_val, y_val , config=val_config)

### sus

In [ ]:
generator = validation_generator
@interact
def select_image_in_batch(batch = (0,len(generator), 1), im = (0,BATCHSIZE- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    global x, y
    x, y = generator.__getitem__(batch)
    info('selected batch : ' + str(batch))
    # logging level == debug --> visualise the generator steps
    info('X shape: {}, Y shape: {}'.format(x.shape, y.shape))
    show_2D_or_3D(x[im], y[im], interpol='bilinear',dpi=100,f_size=(5,5))
    plt.show()
    
    plt.hist(x[im].flatten())
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))

In [ ]:
# Load SAX volumes
x_train, y_train, x_val, y_val =  get_trainings_files(data_path=DATA_PATH_SAX,path_to_folds_df=DF_FOLDS, fold=FOLD)
logging.info('SAX train CMR: {}, SAX train masks: {}'.format(len(x_train), len(y_train)))
logging.info('SAX val CMR: {}, SAX val masks: {}'.format(len(x_val), len(y_val)))

# create a batch generator
batch_generator = DataGenerator(x_train, y_train, config=config)
val_config = config.copy()
val_config['AUGMENT_GRID'] = False# make sure no augmentation will be applied to the validation data
val_config['AUGMENT'] = False
val_config['HIST_MATCHING'] = False
validation_generator = DataGenerator(x_val, y_val , config=val_config)

## Neural Network

In [ ]:
# Create Neural Network for SAX as Unet, changed from PhaseRegressionModel
import src.models.Unets as modelmanager
# create a model
logging.info('Create model')
model = modelmanager.create_unet(config)
model.load_weights(os.path.join(config['MODEL_PATH'],'model.h5'))
logging.info('loaded model weights as h5 file')
model.summary()
 

## Prediction

In [ ]:
import SimpleITK as sitk
gts = stack[0]
preds = stack[1]

# upper = 1, lower == 2
# transform to int representation (one-hot)
gts_flat = np.zeros((gts.shape[:-1]))
gts_flat[gts[...,0]>0.5] = 1
gts_flat[gts[...,1]>0.5] = 2

preds_flat = np.zeros((gts.shape[:-1]))
preds_flat[preds[...,0]>0.5] = 1
preds_flat[preds[...,1]>0.5] = 2
print(preds_flat.shape)
gt_sitks = [sitk.GetImageFromArray(elem) for elem in gts_flat]
pred_sitks = [sitk.GetImageFromArray(elem) for elem in preds_flat]

In [ ]:
import SimpleITK as sitk
gts = stack[0]
preds = stack[1]

# upper = 1, lower == 2
# transform to int representation (one-hot)
gts_flat = np.zeros((gts.shape[:-1]))
gts_flat[gts[...,0]>0.5] = 1
gts_flat[gts[...,1]>0.5] = 2

preds_flat = np.zeros((gts.shape[:-1]))
preds_flat[preds[...,0]>0.5] = 1
preds_flat[preds[...,1]>0.5] = 2
print(preds_flat.shape)
print(preds_flat)
gt_sitks = [sitk.GetImageFromArray(elem) for elem in gts_flat]
pred_sitks = [sitk.GetImageFromArray(elem) for elem in preds_flat]

In [ ]:
import SimpleITK as sitk
gts = stack[0]
preds = stack[1]

# upper = 1, lower == 2
# transform to int representation (one-hot)
gts_flat = np.zeros((gts.shape[:-1]))
gts_flat[gts[...,0]>0.5] = 1
gts_flat[gts[...,1]>0.5] = 2

preds_flat = np.zeros((gts.shape[:-1]))
preds_flat[preds[...,0]>0.5] = 1
preds_flat[preds[...,1]>0.5] = 2
print(preds_flat.shape)
gt_sitks = [sitk.GetImageFromArray(elem) for elem in gts_flat]
pred_sitks = [sitk.GetImageFromArray(elem) for elem in preds_flat]

In [ ]:
import SimpleITK as sitk
gts = stack[0]
preds = stack[1]

# upper = 1, lower == 2
# transform to int representation (one-hot)
gts_flat = np.zeros((gts.shape[:-1]))
gts_flat[gts[...,0]>0.5] = 1
gts_flat[gts[...,1]>0.5] = 2

preds_flat = np.zeros((gts.shape[:-1]))
preds_flat[preds[...,0]>0.5] = 1
preds_flat[preds[...,1]>0.5] = 2
print(preds_flat.shape)
gt_sitks = [sitk.GetImageFromArray(elem) for elem in gts_flat]
pred_sitks = [sitk.GetImageFromArray(elem) for elem in preds_flat]

In [ ]:
# predict on the validation generator
preds = model.predict(validation_generator)
logging.info(preds.shape)

In [ ]:
gts_cmr = np.stack([np.squeeze(x) for x, y in validation_generator])
logging.info(gts_cmr.shape)

In [ ]:
# get all ground truth vectors
gts = np.stack([np.squeeze(y) for x, y in validation_generator])
logging.info(gts.shape)

In [ ]:
df = df[df['fold']==0]

In [ ]:
# duplicate of code that is further on top
for p in sorted(df['patient'].unique()):
    print(p) #prints patient
    files_ = df[df['patient'] ==p]['x_path'].values
    print(len(files_))
    ed_f = files_[:len(files_)//2] # first half of slices are ed
    es_f = files_[len(files_)//2:] # second half of slices are es
    print(len(ed_f)) #prints amount of slices for ED.
    print(len(es_f))

In [ ]:
import pandas as pd
df = pd.read_csv(DF_FOLDS)

In [ ]:
df.head()

In [ ]:
df['patient']

In [ ]:
# duplicate of code that is further on top
for p in sorted(df['patient'].unique()):
    print(p) #prints patient
    files_ = df[df['patient'] ==p]['x_path'].values
    print(len(files_))
    ed_f = files_[:len(files_)//2] # first half of slices are ed
    es_f = files_[len(files_)//2:] # second half of slices are es
    print(len(ed_f)) #prints amount of slices for ED.
    print(len(es_f))

In [ ]:
# predict on the validation generator
preds = model.predict(batch_generator)
logging.info(preds.shape)

In [ ]:
# write 3d_nrrd file to disk
sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_temp_cmr.nrrd')

In [ ]:
# write 3d_nrrd file to disk
sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_flat[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_temp_gt.nrrd')

In [ ]:
# write 3d_nrrd file to disk
sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_temp.nrrd')

In [ ]:
#np.load('/mnt/ssd/git/wft21_septum_landmark_detection/{}'.format(pred_filename))

stack = np.load('/mnt/ssd/git/wft21_septum_landmark_detection/{}'.format(pred_filename))
print('the shape of the numpy.stack is' + str(np.shape(stack)))
print(('loading from: /mnt/ssd/git/wft21_septum_landmark_detection/{}'.format(pred_filename)))

#for-loop over stack --> reading numpy and transferring with pynrrd. pad_and_crop function applied. saving with the name of original file.

In [ ]:
pred_path = os.path.join(config['EXP_PATH'], 'pred_'+ str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")))
ensure_dir(pred_path)
pred_filename = os.path.join(pred_path, 'gtpred_fold{}.npy'.format(config['FOLD'])) #names file according to fold
np.save(pred_filename, np.stack([gts, preds], axis=0))
logging.info('saved as: \n{} \ndone!'.format(pred_filename))

In [ ]:
pred_path = os.path.join(config['EXP_PATH'], 'pred_'+ str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")))
ensure_dir(pred_path)
pred_filename = os.path.join(pred_path, 'gtpred_fold{}.npy'.format(config['FOLD'])) #names file according to fold
np.save(pred_filename, np.stack([gts, preds], axis=0))
logging.info('saved as: \n{} \ndone!'.format(pred_filename))

## Visualize

In [ ]:
from src.visualization.Visualize import show_2D_or_3D
@interact
def show_predictions(elem = (0,len(gts_flat)-1)):
    elem = gts_flat[elem]
    print(elem.shape)
    show_2D_or_3D(elem)

In [ ]:
from src.visualization.Visualize import show_2D_or_3D
@interact
def show_predictions(elem = (0,gts.shape[0])):
    temp = gts_cmr[elem]
    msk = gts_flat[elem]
    print(msk.shape)
    #plt.imshow(temp, cmap='gray')
    plt.imshow(msk, alpha=0.8)
    #show_2D_or_3D(temp, msk)

In [ ]:
from src.visualization.Visualize import show_2D_or_3D
@interact
def show_predictions(elem = (0,len(preds_flat)-1)):
    elem = gts_flat[elem]
    print(elem.shape)
    show_2D_or_3D(elem)

In [ ]:
from src.visualization.Visualize import show_2D_or_3D
@interact
def show_predictions(elem = (0,gts.shape[0])):
    temp = gts_cmr[elem]
    msk = preds_flat[elem]
    print(msk.shape)
    #plt.imshow(temp, cmap='gray')
    #plt.imshow(msk, alpha=0.8)
    show_2D_or_3D(temp, msk)

In [ ]:
from src.visualization.Visualize import show_2D_or_3D
@interact
def show_predictions(elem = (0,gts.shape[0])):
    temp = gts_cmr[elem]
    msk = gts_flat[elem]
    print(msk.shape)
    #plt.imshow(temp, cmap='gray')
    #plt.imshow(msk, alpha=0.8)
    show_2D_or_3D(temp, msk)